<a href="https://colab.research.google.com/github/as9786/ML-DLPratice/blob/main/%EC%98%A4%EC%B0%A8%EC%97%AD%EC%A0%84%ED%8C%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 수치 미분을 이용한 심층 신경망 학습

In [1]:
import time
import numpy as np

# Utility function

In [2]:
epsilon = 0.0001

def _t(x):
  return np.transpose(x)

def _m(A,B):
  return np.matmul(A,B)

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def mean_squared_error(h,y):
  return 1 / 2 * np.mean(np.square(h-y))

# 신경망 구현

In [17]:
class Neuron:
  def __init__(self,W,b,a):
    self.W = W
    self.b = b
    self.a = a 

    # 경사
    self.dw = np.zeros_like(self.W)
    self.db = np.zeros_like(self.b)
  
  def __call__(self,x):
    return self.a(_m(_t(self.W),x)+self.b) # activation((W^T)x + b)

# 심층 신경망 구현

In [21]:
class DNN:
  def __init__(self,hidden_depth,num_neuron,num_input,num_output,activation=sigmoid):
    def init_var(i,o):
      return np.random.normal(0.0,0.01,(i,o)),np.zeros((o,))

    self.sequence = list()
    # 첫 은닉층
    W,b = init_var(num_input,num_neuron)
    self.sequence.append(Neuron(W,b,activation))

    # 은닉층들
    for _ in range(hidden_depth-1):
      W,b = init_var(num_neuron,num_neuron)
      self.sequence.append(Neuron(W,b,activation))
    
    # 출력층
    W,b = init_var(num_neuron,num_output)
    self.sequence.append(Neuron(W,b,activation))
  
  def __call__(self,x):
    for layer in self.sequence:
      x = layer(x)
    return x 

  def calc_gradient(self,x,y,loss_func):
    def get_new_sequence(layer_index,new_neuron):
      new_sequence = list()
      for i, layer in enumerate(self.sequence):
        if i == layer_index:
          new_sequence.append(new_neuron)
        else:
          new_sequence.append(layer)
      return new_sequence

    def eval_sequence(x,sequence):
      for layer in sequence:
        x = layer(x)
      return x

    loss = loss_func(self(x),y)

    for layer_id, layer in enumerate(self.sequence): #iterate layer
      for w_i,w in enumerate(layer.W): # iterate W(row)
        for w_j, ww in enumerate(w): # iterate W (col)
          W = np.copy(layer.W)
          W[w_i][w_j] = ww + epsilon 

          new_neuron = Neuron(W,layer.b,layer.a)
          new_seq = get_new_sequence(layer_id,new_neuron)
          h = eval_sequence(x,new_seq) 

          num_grad = (loss_func(h,y)-loss) / epsilon #(f(x+eps)-f(x))/epsilon
          layer.dw[w_i][w_j] = num_grad

        for b_i, bb in enumerate(layer.b): # iterate b
          b = np.copy(layer.b)
          b[b_i] = bb + epsilon 

          new_neuron = Neuron(W,layer.b,layer.a)
          new_seq = get_new_sequence(layer_id,new_neuron)
          h = eval_sequence(x,new_seq) 

          num_grad = (loss_func(h,y)-loss) / epsilon #(f(x+eps)-f(x))/epsilon
          layer.db[b_i] = num_grad

    return loss

# 경사하강법

In [24]:
def gradient_descent(network,x,y,loss_obj,alpha=0.01):

  loss = network.calc_gradient(x,y,loss_obj)
  for layer in network.sequence:
    layer.W += -alpha * layer.dw
    layer.b += -alpha * layer.db
  return loss

# 시험

In [25]:
x = np.random.normal(0.0,1.0,(10,))
y = np.random.normal(0.0,1.0,(2,))

dnn = DNN(hidden_depth=5,num_neuron=32,num_input=10,num_output=2,activation=sigmoid)

t = time.time()

for epoch in range(100):
  loss = gradient_descent(dnn,x,y,mean_squared_error,0.01)
  print('Epoch : {} Test loss : {}'.format(epoch,loss))

print('{} seconds elapsed'.format(time.time()-t))

Epoch : 0 Test loss : 0.5342294150442677
Epoch : 1 Test loss : 0.5314448482373073
Epoch : 2 Test loss : 0.5286751831098883
Epoch : 3 Test loss : 0.5259206902809807
Epoch : 4 Test loss : 0.5231816302558056
Epoch : 5 Test loss : 0.5204582533695035
Epoch : 6 Test loss : 0.517750799746941
Epoch : 7 Test loss : 0.5150594992780231
Epoch : 8 Test loss : 0.5123845716086496
Epoch : 9 Test loss : 0.5097262261463181
Epoch : 10 Test loss : 0.5070846620805649
Epoch : 11 Test loss : 0.5044600684172731
Epoch : 12 Test loss : 0.5018526240265765
Epoch : 13 Test loss : 0.4992624977041744
Epoch : 14 Test loss : 0.4966898482446226
Epoch : 15 Test loss : 0.4941348245273312
Epoch : 16 Test loss : 0.49159756561339096
Epoch : 17 Test loss : 0.48907820085371473
Epoch : 18 Test loss : 0.48657685000737505
Epoch : 19 Test loss : 0.48409362336936634
Epoch : 20 Test loss : 0.4816286219078043
Epoch : 21 Test loss : 0.47918193740962767
Epoch : 22 Test loss : 0.4767536526338325
Epoch : 23 Test loss : 0.474343841472531